In [2]:
import requests
import pandas as pd
import urllib3
import json
import datetime


def string_to_dict(dict_string):
    # Convert to proper json format
    dict_string = dict_string.replace("T00:00:00", '')
    tmp_dict = json.loads(dict_string)
    new_ord = ['date','close','stockAgentMainPower','stockAgentDiff','skp5','skp20']
    ret_dict = {}
    for k in new_ord:
        if(k=='date'):
            data = datetime.datetime.strptime(tmp_dict.get(k), '%Y-%m-%d')
        else:
            data = tmp_dict.get(k)
        ret_dict.update({k:data})
    return ret_dict


def download_major_investors(stockid):
    urllib3.disable_warnings()
    url = 'https://www.wantgoo.com/stock/%s/major-investors/main-trend-data' %stockid
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'}

    
    r = requests.get(url, headers=headers)
    a = r.text[1:len(r.text)-1]
    tmp_list = []
    while(len(a)>0):
        n = a.find('},')
        if(n>0):
            b = a[:n+1]
            tmp_list.append(b)
            c = a[n+2:]
            a = c
        else:
            n=a.find('}')
            b = a[:n+1]
            tmp_list.append(b)
            c = a[n+2:]
            a = c

    major_list=[]
    for ls in tmp_list:
        major_list.append(string_to_dict(ls))

    cols = {'date':'日期','close':'收盤價','stockAgentMainPower':'買賣超','stockAgentDiff':'家數差','skp5':'5日集中(%)','skp20':'20日集中(%)'}
    df = pd.DataFrame(major_list)
    df.rename(columns=cols, inplace=True)
    df.set_index('日期', inplace=True)
    return df

def main():
    mystock='0050'
    print('股票代碼：', mystock)
    dfmajor_investors=download_major_investors(mystock)
    display(dfmajor_investors)


if __name__ == "__main__":
    main()

股票代碼： 0050


,收盤價,買賣超,家數差,5日集中(%),20日集中(%)
日期,,,,,
2020-11-05,106.05,1193,-201,-23.3440,2.8647
2020-11-04,106.15,1587,-277,-37.3455,3.0994
2020-11-03,104.60,-589,156,-44.8521,4.3641
2020-11-02,103.40,-5007,638,-47.2768,5.8290
2020-10-30,103.00,-3527,610,-40.8573,12.9902
...,...,...,...,...,...
2019-11-12,92.00,236,-51,7.9487,15.6745
2019-11-11,91.15,270,33,6.2017,17.0877
2019-11-08,92.55,-45,45,13.3948,19.8048
